# Plotting with `gu_toolkit`: a comprehensive tour

This notebook walks through the most common plotting workflows you will use with the toolkit.
We focus on *clean math exploration patterns* that work well in teaching, research, and quick exploratory analysis.

By the end you will see how to:

- Build plots from SymPy expressions.
- Layer multiple traces on one figure.
- Add interactive parameters (sliders) automatically or manually.
- Customize ranges, sampling density, and styling.
- Use global `plot(...)` and figure contexts for concise demos.
- Create lightweight *info panels* that respond to parameter changes.
- Prototype common calculus and modeling ideas.

> **Note:** This notebook assumes you are running in Jupyter or JupyterLab so that widgets and Plotly figures render inline.


## Setup

We use SymPy for symbolic expressions and `SmartFigure` for interactive plotting. The global helpers
`plot`, `params`, and `parameter` are convenient for rapid prototyping.


In [1]:
import sympy as sp

from pathlib import Path
import sys
ROOT = Path.cwd().resolve().parents[1]  
sys.path.insert(0, str(ROOT))
from gu_toolkit import *


## 1. Your first plot (single trace)

`SmartFigure` accepts SymPy expressions directly. The figure auto-compiles them to NumPy and renders them
with Plotly.


In [2]:
fig = SmartFigure(x_range=(-6, 6), y_range=(-2.5, 2.5))
fig.title = "Sine wave"
display(fig)
with fig:
    plot(x, sin(x), id="sin")


OneShotOutput()

## 2. Layering multiple traces

Add multiple plots to the same figure to compare functions. If you reuse an `id`, the trace is updated
instead of replaced.


In [3]:
fig2 = SmartFigure(x_range=(-6, 6), y_range=(-3, 3))
fig2.title = "Multiple traces"
fig2
with fig2:
    plot(x, sp.sin(x), id="sin")
    plot(x, sp.cos(x), id="cos")
    plot(x, sp.sin(2 * x), id="sin2", dash="dash", color="#d62728")


ValueError: dictionary update sequence element #0 has length 5; 2 is required

## 3. Automatic parameters (sliders)

When an expression contains symbols besides the plot variable, `SmartFigure` automatically creates sliders
for them. The parameter symbols are inferred from the expression.


In [4]:
fig3 = SmartFigure(x_range=(-6, 6), y_range=(-3, 3))
fig3.title = "Auto-created parameters"
display(fig3)
with fig3:
    plot(x, a * sp.sin(x), id="a_sin")
    plot(x, sp.cos(x + b), id="b_shift")


OneShotOutput()

### Adjusting slider defaults

You can control slider ranges and defaults by calling `parameter` directly. This is especially helpful
when you want a parameter to start at a specific value or use a specific range/step size.


In [5]:
with fig3:
    parameter(a, min=-2, max=2, value=1, step=0.1)
    parameter(b, min=-3.14, max=3.14, value=0.0, step=0.05)


In [6]:
with fig3:
    parameter(a, min=3, max=4, value=3.5, step=0.1)
    parameter(b, min=-3.14, max=3.14, value=0.0, step=0.05)


## 4. Explicit parameter control

You can specify parameters before plotting to avoid creating them with defaults.


In [8]:
fig4 = SmartFigure(x_range=(-4, 4), y_range=(-5, 5))
fig4.title = "Explicit parameters"
display(fig4)

# Explicit parameter list: introduce parameters before the plot
with fig4:
    parameter(a, min=0.1, max=3.0, value=1.0, step=0.1)
    parameter(b, min=0.1, max=3.0, value=1.0, step=0.1)
    plot(x, a * sp.exp(-x**2) + b, id="gaussian")

# Change limits
with fig4:
    parameter(a, min=4, max=5.0, value=4.1, step=0.1)
    parameter(b, min=0.1, max=3.0, value=1.0, step=0.1)


OneShotOutput()

TypeError: 'NoneType' object is not subscriptable

## 5. Customizing the domain and sampling density

Use `x_domain` (per-trace) and `sampling_points` to refine how curves are sampled. This is especially
useful for rapidly oscillating functions.


In [11]:
fig5 = SmartFigure(x_range=(-1, 1), y_range=(-2, 2), sampling_points=200)

fig5.plot(x, sp.sin(15 * x), id="dense", color="#1f77b4")
fig5.plot(x, sp.sin(15 * x), id="dense_zoomed", x_domain=(-0.5, 0.5), sampling_points=600, dash="dot", color="#ff7f0e")
fig5.title = "Sampling and domain control"
fig5


ValueError: dictionary update sequence element #0 has length 5; 2 is required

## 6. Styling traces

The `plot` method accepts common styling arguments. You can pass `color`, `thickness`, `dash`, or even
full Plotly `line`/`trace` dictionaries for advanced settings.


In [ ]:
fig6 = SmartFigure(x_range=(-6, 6), y_range=(-4, 4))

fig6.plot(x, sp.sin(x), id="base", color="#2ca02c", thickness=3)
fig6.plot(x, sp.sin(x) + 1, id="shifted", dash="dash", line={"width": 2, "color": "#9467bd"})
fig6.plot(x, sp.sin(x) - 1, id="glow", trace={"opacity": 0.6})
fig6.title = "Styling examples"
fig6


## 7. Global `plot(...)` + figure contexts

If you prefer a *very* terse workflow, you can use the global `plot(...)` function.
Inside a figure context (`with fig:`), each call routes to the current figure.


In [ ]:
fig7 = SmartFigure(x_range=(-6, 6), y_range=(-2.5, 2.5))
with fig7:
    plot(x, sp.sin(x), id="sin")
    plot(x, sp.cos(x), id="cos", dash="dash")
    plot(x, sp.sin(x + a), id="shifted")

fig7.parameter(a, min=-3.14, max=3.14, value=0.0, step=0.05)
fig7.title = "Global plot with context"
fig7


### Using the `params` proxy

`params` is a global proxy that refers to the current figure (when used in a context). It provides
quick access to parameter values.


In [ ]:
with fig7:
    params[a].value = 1.0

fig7


## 8. Calculus exploration: tangent line + local linearization

A common exploration is to visualize a function with its tangent line at a chosen point. Here we
build an expression for the tangent line and let `a` be the point of tangency.


In [ ]:
f = sp.sin(x) + 0.2 * x

# Tangent line at x = a
fprime = sp.diff(f, x)
tangent = f.subs(x, a) + fprime.subs(x, a) * (x - a)

fig8 = SmartFigure(x_range=(-6, 6), y_range=(-4, 4))
fig8.plot(x, f, id="f")
fig8.plot(x, tangent, id="tangent", dash="dash", color="#d62728")
fig8.parameter(a, min=-5, max=5, value=0, step=0.1)
fig8.title = "Tangent line exploration"
fig8


## 9. Modeling: damped oscillations

Let \(a\) control the decay rate and \(b\) control the frequency. This is a classic exploration for
differential equations and signal processing.


In [ ]:
fig9 = SmartFigure(x_range=(0, 10), y_range=(-2, 2))
expr = sp.exp(-a * x) * sp.cos(b * x)
fig9.plot(x, expr, id="damped")
fig9.parameter(a, min=0.0, max=1.0, value=0.2, step=0.02)
fig9.parameter(b, min=0.5, max=8.0, value=2.0, step=0.1)
fig9.title = "Damped oscillation"
fig9


## 10. Parametric comparisons: family of polynomials

A simple but effective exploration is to vary coefficients and see how roots and shapes change.
Here we use a cubic \(x^3 + ax^2 + bx + c\).


In [ ]:
fig10 = SmartFigure(x_range=(-3, 3), y_range=(-10, 10))
poly = x**3 + a * x**2 + b * x + c
fig10.plot(x, poly, id="poly")

fig10.parameter(a, min=-3, max=3, value=0.0, step=0.1)
fig10.parameter(b, min=-3, max=3, value=0.0, step=0.1)
fig10.parameter(c, min=-3, max=3, value=0.0, step=0.1)
fig10.title = "Cubic family"
fig10


## 11. Info panel: live text outputs

`SmartFigure` includes an **Info** sidebar for small textual summaries. We can show a computed value
(e.g., function value or slope) whenever parameters change.


In [ ]:
fig11 = SmartFigure(x_range=(-5, 5), y_range=(-4, 4))
expr = sp.sin(x) + 0.1 * x**2
fig11.plot(x, expr, id="expr")

fig11.parameter(a, min=-4, max=4, value=1.0, step=0.1)

info_out = fig11.get_info_output("summary")

# Define a simple hook to update the info panel when params change.
def update_summary(event):
    value = float(expr.subs(x, fig11.params[a].value))
    slope = float(sp.diff(expr, x).subs(x, fig11.params[a].value))
    with info_out:
        info_out.clear_output()
        print(f"x = {fig11.params[a].value:.2f}")
        print(f"f(x) = {value:.3f}")
        print(f"f'(x) = {slope:.3f}")

fig11.add_param_change_hook(update_summary, run_now=True)
fig11.title = "Info panel with live summary"
fig11


## 12. Updating plots in-place

When you reuse a plot `id`, `SmartFigure` updates the existing trace. This is useful when you
want to quickly swap out the expression or change styling.


In [ ]:
fig12 = SmartFigure(x_range=(-4, 4), y_range=(-4, 4))

fig12.plot(x, sp.sin(x), id="f", color="#1f77b4")
fig12.title = "In-place updates"
fig12


In [ ]:
# Update the same plot id with a new function and style.
fig12.plot(x, sp.cos(x), id="f", color="#ff7f0e", thickness=3, dash="dash")
fig12


## 13. Quick math exploration ideas (grab-and-go)

Below are short snippets you can copy/paste as starting points for common explorations.


In [ ]:
# Logistic growth
fig13 = SmartFigure(x_range=(0, 10), y_range=(0, 1.2))
logistic = 1 / (1 + sp.exp(-a * (x - b)))
fig13.plot(x, logistic, id="logistic")
fig13.parameter(a, min=0.1, max=4.0, value=1.5, step=0.1)
fig13.parameter(b, min=0, max=10, value=5, step=0.1)
fig13.title = "Logistic growth"
fig13


In [ ]:
# Beating waves
fig14 = SmartFigure(x_range=(0, 20), y_range=(-2, 2))
expr = sp.sin(a * x) + sp.sin(b * x)
fig14.plot(x, expr, id="beating")
fig14.parameter(a, min=0.5, max=3.0, value=1.0, step=0.05)
fig14.parameter(b, min=0.5, max=3.0, value=1.2, step=0.05)
fig14.title = "Beating waves"
fig14


## 14. Wrap-up

You now have a toolkit for: 

- building interactive plots from symbolic expressions,
- managing parameters with sliders,
- controlling sampling and appearance,
- and adding small live-readouts for context.

Try remixing these patterns for your own models and lessons!
